In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Irritable_bowel_syndrome_(IBS)"
cohort = "GSE25220"

# Input paths
in_trait_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)"
in_cohort_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)/GSE25220"

# Output paths
out_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE25220.csv"
out_gene_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE25220.csv"
out_clinical_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE25220.csv"
json_path = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcription levels in human colon biopsies in IBS and IBD patients before and after participating in a high red-meat dietary intervention"
!Series_summary	"Study 1: Transcriptomic profiles in colon tissue from inflammatory bowel diseases patients in relation to N-nitroso compound exposure and colorectal cancer risk"
!Series_summary	""
!Series_summary	"Study 1: N-nitroso compounds (NOC) have been suggested to play a role in human cancer development but definitive evidence is still lacking. In this study we investigated gene expression modifications induced in human colon tissue in relation to NOC exposure to gain insight in the relevance of these compounds in human colorectal cancer (CRC) development. Since there are indications that inflammation stimulates endogenous NOC formation, the study population consisted of patients with inflammatory bowel disease (IBD) and irritable bowel syndrome patients as controls without inflammation. Strong transc

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptomic data (gene expression)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: Disease status is available at index 1 (IBD vs IBS)
trait_row = 1

# For age: Age is available at index 3
age_row = 3

# For gender: Gender is available at index 2
gender_row = 2

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert trait value to binary (0 for IBS, 1 for IBD)
    Since we're studying IBS, the coding should be:
    - IBS (control/our trait of interest) = 1
    - IBD (different condition) = 0
    """
    if value is None:
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":")[1].strip()
    
    if value.upper() == "IBS":
        return 1  # Our trait of interest
    elif value.upper() == "IBD":
        return 0  # Not our trait of interest
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if value is None:
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":")[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":")[1].strip()
    
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(clinical_selected)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_selected.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM620750': [0.0, 57.0, 1.0], 'GSM620751': [0.0, 57.0, 1.0], 'GSM620752': [1.0, 65.0, 0.0], 'GSM620753': [1.0, 65.0, 0.0], 'GSM620754': [1.0, 44.0, 1.0], 'GSM620755': [0.0, 72.0, 1.0], 'GSM620756': [0.0, 60.0, 1.0], 'GSM620757': [0.0, 60.0, 1.0], 'GSM620758': [1.0, 57.0, 1.0], 'GSM620759': [1.0, 57.0, 1.0], 'GSM620760': [1.0, 25.0, 0.0], 'GSM620761': [1.0, 25.0, 0.0], 'GSM620762': [0.0, 31.0, 1.0], 'GSM620763': [0.0, 31.0, 1.0], 'GSM620764': [0.0, 48.0, 0.0], 'GSM620765': [0.0, 48.0, 0.0], 'GSM620766': [0.0, 54.0, 1.0], 'GSM620767': [0.0, 71.0, 1.0], 'GSM620768': [0.0, 71.0, 1.0], 'GSM620769': [1.0, 63.0, 0.0], 'GSM620770': [0.0, 29.0, 1.0], 'GSM620771': [0.0, 29.0, 1.0], 'GSM620772': [1.0, 61.0, 1.0], 'GSM620773': [1.0, 61.0, 1.0], 'GSM620774': [1.0, 39.0, 0.0], 'GSM620775': [1.0, 65.0, 1.0], 'GSM620776': [0.0, 55.0, 0.0], 'GSM620777': [1.0, 69.0, 1.0], 'GSM620778': [1.0, 65.0, 0.0], 'GSM620779': [0.0, 53.0, 1.0], 'GSM620780': [0.0, 40.0, 0.0], 'GSM620781': [0

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 83
Header line: "ID_REF"	"GSM620750"	"GSM620751"	"GSM620752"	"GSM620753"	"GSM620754"	"GSM620755"	"GSM620756"	"GSM620757"	"GSM620758"	"GSM620759"	"GSM620760"	"GSM620761"	"GSM620762"	"GSM620763"	"GSM620764"	"GSM620765"	"GSM620766"	"GSM620767"	"GSM620768"	"GSM620769"	"GSM620770"	"GSM620771"	"GSM620772"	"GSM620773"	"GSM620774"	"GSM620775"	"GSM620776"	"GSM620777"	"GSM620778"	"GSM620779"	"GSM620780"	"GSM620781"	"GSM620782"	"GSM620783"	"GSM620784"	"GSM620785"	"GSM620786"	"GSM620787"	"GSM620788"	"GSM620789"	"GSM620790"	"GSM620791"	"GSM620792"	"GSM620793"
First data line: "(+)E1A_r60_1"
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(+)eQC-39', '(+)eQC-40',
       '(+)eQC-41', '(+)eQC-42', '(-)3xSLv1', 'A_23_P100001', 'A_23_P100011',
       'A_23_P100022', 'A_23_P100056', 'A_23_P100074'],
      dtype='object', name='ID

### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# Example identifiers: '(+)E1A_r60_1', '(+)E1A_r60_3', 'A_23_P100001', 'A_23_P100011', etc.
# These appear to be Agilent microarray probe identifiers, not standard human gene symbols
# Agilent platform IDs like 'A_23_P100001' need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns that store gene identifiers and gene symbols
# Based on the preview, we need to map from 'ID' to 'GENE_SYMBOL'
probe_col = "ID"  # Column that contains the probe identifiers from gene expression data
gene_symbol_col = "GENE_SYMBOL"  # Column that contains the gene symbols

# 2. Create the gene mapping dataframe using the function from the library
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_symbol_col)

# Display the first few rows of the mapping dataframe to confirm
print("Gene mapping preview:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Display preview of the gene expression data
print("\nGene expression data preview:")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {len(gene_data.columns)}")
print("First 5 genes:")
print(gene_data.index[:5])

# Save the gene expression data to CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'Gene': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN']}

Gene expression data preview:
Number of genes: 18488
Number of samples: 44
First 5 genes:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE25220.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First load the gene expression data
gene_data = pd.read_csv(out_gene_data_file, index_col=0)

# Normalize gene symbols using the NCBI Gene database
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data)} genes remain")

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Load the clinical data
clinical_data = pd.read_csv(out_clinical_data_file)

# Ensure proper column naming for clinical data
if clinical_data.columns[0] == "Unnamed: 0":
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)

# Inspect what's in the clinical data
print("Clinical data columns:", clinical_data.columns.tolist())
print("Clinical data shape:", clinical_data.shape)
print("Clinical data sample:", clinical_data.head())

# 3. Link the clinical and genetic data
# Transpose gene data to have samples as rows and genes as columns
gene_data_t = gene_data.T

# Make sure the trait column exists in clinical data
# If not, we need to load and process it again
if trait not in clinical_data.columns:
    print(f"Trait column '{trait}' not found in clinical data. Loading raw clinical data...")
    
    # Load raw data again
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, raw_clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Process disease state row manually to ensure correct trait column
    disease_row = raw_clinical_data.iloc[trait_row]
    samples = [col for col in disease_row.index if col != "!Sample_geo_accession"]
    
    # Create a DataFrame for the trait values
    trait_data = pd.DataFrame(index=samples)
    
    # Process each sample's disease status
    for sample in samples:
        value = disease_row[sample]
        if pd.isna(value):
            trait_data.loc[sample, trait] = None
        else:
            if ":" in value:
                value = value.split(":", 1)[1].strip()
            
            if "IBS" in value:
                trait_data.loc[sample, trait] = 1  # IBS is our target trait
            elif "IBD" in value:
                trait_data.loc[sample, trait] = 0  # IBD is the control
            else:
                trait_data.loc[sample, trait] = None
    
    clinical_data = trait_data
    print("Recreated clinical data with correct trait column")

# Find common samples between clinical and gene data
common_samples = list(set(clinical_data.index) & set(gene_data_t.index))
print(f"Found {len(common_samples)} common samples between clinical and genetic data")

if len(common_samples) > 0:
    # Filter both datasets to include only common samples
    gene_data_filtered = gene_data_t.loc[common_samples]
    clinical_data_filtered = clinical_data.loc[common_samples]
    
    # Join the data
    linked_data = pd.concat([clinical_data_filtered, gene_data_filtered], axis=1)
    print(f"Linked data shape: {linked_data.shape}")
    print(f"Linked data columns preview: {linked_data.columns[:5].tolist()}")
else:
    print("No common samples found. Unable to link data.")
    # We can't proceed without linked data
    is_usable = False
    # Log this in the cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Set to True as we can't use this data
        df=pd.DataFrame(),  # Empty DataFrame
        note="Unable to link clinical and genetic data due to no common sample IDs."
    )
    # Exit early
    raise ValueError("No common samples found between clinical and gene data. Cannot proceed.")

# Verify trait column exists in linked data
if trait not in linked_data.columns:
    print(f"ERROR: Trait column '{trait}' still not found in linked data")
    # Create a simple mapping if trait column missing
    linked_data[trait] = clinical_data_filtered[trait]

print(f"Final linked data columns (first 10): {linked_data.columns[:10].tolist()}")
print(f"Trait column exists: {trait in linked_data.columns}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Check if trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation and save relevant information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from patients with IBS and IBD, examining transcriptomic responses."
)

# 7. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

After normalization: 18247 genes remain


Normalized gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE25220.csv
Clinical data columns: ['GSM620750', 'GSM620751', 'GSM620752', 'GSM620753', 'GSM620754', 'GSM620755', 'GSM620756', 'GSM620757', 'GSM620758', 'GSM620759', 'GSM620760', 'GSM620761', 'GSM620762', 'GSM620763', 'GSM620764', 'GSM620765', 'GSM620766', 'GSM620767', 'GSM620768', 'GSM620769', 'GSM620770', 'GSM620771', 'GSM620772', 'GSM620773', 'GSM620774', 'GSM620775', 'GSM620776', 'GSM620777', 'GSM620778', 'GSM620779', 'GSM620780', 'GSM620781', 'GSM620782', 'GSM620783', 'GSM620784', 'GSM620785', 'GSM620786', 'GSM620787', 'GSM620788', 'GSM620789', 'GSM620790', 'GSM620791', 'GSM620792', 'GSM620793']
Clinical data shape: (3, 44)
Clinical data sample:    GSM620750  GSM620751  GSM620752  GSM620753  GSM620754  GSM620755  \
0        0.0        0.0        1.0        1.0        1.0        0.0   
1       57.0       57.0       65.0       65.0       44.0       72.0   
2        1.0        1

Data after handling missing values: (44, 18248)
For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '1.0' with 19 occurrences. This represents 43.18% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.



Linked data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE25220.csv
